# Assignment 2

__41934 - Advanced BIM, E23__


Kaare G. S. Hansen, s214282 - DTU




BIM ideer:
- Byg etager oven på hindanden, forstør søjler etc...
- Ændre nyttelast i spaces, lav ny geometri, lav FEM-model
- Opennest, produktionstegninger

## Imports

In [1]:
import os
import sys
from collections import namedtuple
import numpy as np
import pandas as pd
import ifcopenshell
import ifcopenshell.util.placement
import ifcopenshell.geom
import ifcopenshell.util.shape
import ifcopenshell.util.selector

print(f"ifcopenshell version: {ifcopenshell.version}")
print(f"Python-version {sys.version}")

ifcopenshell version: 0.7.0
Python-version 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [2]:
from geomUtils import *
from ifcUtils import *

In [3]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = "png"
pio.renderers.default = "plotly_mimetype+png"

## Code

### Load models

In [4]:
models = load_models()

File path, ark: /Users/Kaare/My Drive/DTU/Kurser/Videregaaende BIM - 41934/IFC-models\SkyLab/LLYN - ARK.ifc
Model schema: IFC4

File path, stru: /Users/Kaare/My Drive/DTU/Kurser/Videregaaende BIM - 41934/IFC-models\SkyLab/LLYN - STRU.ifc
Model schema: IFC4



In [5]:
model = models['stru']
# model = models['ark']

### Load bearing elements

In [6]:
load_bearing = getLoadBearing(model)
len(load_bearing)

Number of load-bearing elements: 1750
Unique types of loadbearing elements:
 {'IfcWall', 'IfcSlab', 'IfcBeam', 'IfcColumn'}


1750

In [7]:
import OCC.Core
import OCC.Core.BRep
import OCC.Core.BRepPrimAPI
import OCC.Core.BRepTools

In [8]:
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_VolumeProperties

## Get IfcBeams as lines

In [9]:
bearingElements = dict()
for elm in load_bearing:
    key = elm.is_a()
    if not key in bearingElements:
        bearingElements[key] = list()
    
    bearingElements[key].append(elm)

[f"{key}: {len(val)}" for key, val in bearingElements.items()]

['IfcBeam: 565', 'IfcColumn: 452', 'IfcSlab: 365', 'IfcWall: 368']

In [10]:
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_PYTHON_OPENCASCADE, True)  # tells ifcopenshell to use pythonocc

#################
# Extra options #
#################

# settings.set(settings.USE_BREP_DATA,True)
# settings.set(settings.SEW_SHELLS,True)
# settings.set(settings.USE_WORLD_COORDS,True)
settings.set(settings.INCLUDE_CURVES, True)
# settings.set(settings.EDGE_ARROWS, True)

In [11]:
def getCurveShapes(elements):
    curveShapes = list()
    for element in elements:
        axis = ifcopenshell.util.representation.get_representation(element, "Model", "Axis")
        assert axis is not None
        curve_3d_rep = ifcopenshell.util.representation.resolve_representation(axis)
        curveShape = ifcopenshell.geom.create_shape(settings, inst=element, repr=curve_3d_rep)

        curveShapes.append(curveShape)
    
    return curveShapes

In [12]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.gp import gp_Pnt, gp_Ax2, gp_Dir, gp_XYZ
from OCC.Core.BRepBndLib import brepbndlib
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from OCC.Core.Bnd import Bnd_OBB
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import TopoDS_Shape
from OCC.Core.BRep import BRep_Builder
from OCC.Core.GC import GC_MakeArcOfCircle, GC_MakeSegment

from OCC.Display.SimpleGui import init_display
from OCC.Display.OCCViewer import rgb_color

from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Lin
from OCC.Core.Geom import Geom_Line, Geom_Point, Geom_CartesianPoint
from OCC.Core.AIS import AIS_Line, AIS_Point
from OCC.Core.Prs3d import Prs3d_LineAspect, Prs3d_Drawer
from OCC.Core.Quantity import Quantity_Color


In [13]:
beams = bearingElements['IfcBeam']
columns = bearingElements['IfcColumn']
slabs = bearingElements['IfcSlab']
walls = bearingElements['IfcWall']
elements = [*beams, *columns, *slabs, *walls]
elementsColWalls = [*beams, *columns]

In [14]:
beamCurves = getCurveShapes(beams)
len(beamCurves)

565

In [15]:
# columnCurves = getCurveShapes(columns)


In [16]:
curveShapes = beamCurves
curveShape = curveShapes[0]
curveShape.geometry

<class 'TopoDS_Compound'>

In [17]:
occ_display, start_display, add_menu, add_function_to_menu = init_display()
for curveShape in [curveShapes[i] for i in range(len(curveShapes))]:
    occ_display.DisplayShape(curveShape.geometry)
    # occ_display.DisplayShape(curveShape, update=True)

# occ_display.DisplayShape(curveShape, update=True)
# occ_display.SetRaytracingMode(depth=3)

occ_display.FitAll()

start_display()

qt-pyqt5 backend - Qt version 5.15.8


In [19]:
occ_display, start_display, add_menu, add_function_to_menu = init_display()

# Display bodies
NumRender = 50
# NumRender = len(beams)
# for product in [beams[i] for i in range(NumRender)]:
# for product in elementsColWalls:
for product in elements:
# for product in [elements[i] for i in range(NumRender)]:
    try:
        if (
            product.Representation is not None
        ):  # some IfcProducts don't have any 3d representation
            # pdct_shape = ifcopenshell.geom.create_shape(settings, inst=product)

            body = ifcopenshell.util.representation.get_representation(product, "Model", "Body")
            body_repr = ifcopenshell.util.representation.resolve_representation(body)
            pdct_shape = ifcopenshell.geom.create_shape(settings, inst=product, repr=body_repr)

            r, g, b, alpha = pdct_shape.styles[0]  # the shape color

            # below, the pdct_shape.geometry is a TopoDS_Shape, i.e. can be rendered using
            # any renderer (threejs, x3dom, jupyter, qt5 based etc.)
            if False:
                occ_display.DisplayShape(
                    # pdct_shape.geometry, shape_color=color, transparency=True, opacity=alpha
                    pdct_shape.geometry,
                    # pdct_shape[1],
                    color=rgb_color(r, g, b),
                    # transparency=alpha,
                    transparency=0.7,
                )

            # Oriented bounding box
            if True:
                elmShape = pdct_shape.geometry
                obb = Bnd_OBB()
                brepbndlib.AddOBB(elmShape, obb, True, True, True)
                obb_shape = convert_bnd_to_shape(obb)
                occ_display.DisplayShape(obb_shape, color=rgb_color(1.0, 0.0, 1.0), transparency=0.8)

                ais_line = convert_bnd_to_line(obb)


                # lineSeg = GC_MakeSegment(pnt, pnt2).Value()
                # ais_line = AIS_Line(lineSeg)


                # pt1 = Geom_CartesianPoint(pnt)
                # pt2 = Geom_CartesianPoint(pnt2)
                # ais_line = AIS_Line(pt1, pt2)
                # print(type(ais_line))
                # print(type(z_dir))
                # line = gp_Lin(pnt, z_dir)

                # ais_pnt = AIS_Point(pnt)
                # ais_line = ais_pnt
                width = float(3.0)
                drawer = ais_line.Attributes()
                ais_line.SetWidth(width)

                # asp : first parameter color, second type, last width
                i = 1
                asp = Prs3d_LineAspect(Quantity_Color(9 * i), i, width*5.0)
                # drawer.SetLineAspect(asp)
                # drawer.SetLineArrowDraw(True)
                ais_line.SetAttributes(drawer)
                # occ_display.DisplayShape(ais_line)
                occ_display.Context.Display(ais_line, False)
                # occ_display.DisplayShape(pnt, color=rgb_color(0.0, 1.0, 0.0))
                # occ_display.DisplayShape(lineSeg, color=rgb_color(1.0, 1.0, 0.0))
                # occ_display.DisplayShape(pnt2, color=rgb_color(1.0, 1.0, 0.0))

        else:
            print(f"Product has no representation!")

    except Exception as e:
        print(f"Error! {e}")

# Display lines
if False:
    for curveShape in [curveShapes[i] for i in range(NumRender)]:
        occ_display.DisplayShape(curveShape.geometry)


# occ_display.DisplayShape(curveShape, update=True)
# occ_display.SetRaytracingMode(depth=1)
occ_display.FitAll()

start_display()

INFO:OCC.Display.backend:The qt-pyqt5 backend is already loaded...``load_backend`` can only be called once per session
qt-pyqt5 backend - Qt version 5.15.8
